# this is to compare same huggingface pipeline task with differnt models
## list of tasks
- sentiment-analysis
- ner
- question-answering
- summarization
- audio-classification
- automatic-speech-recognition
- image-classification
- object-detection

In [ ]:
# to install those depdencies once only
#%pip install --upgrade transformers accelerate torch torchvision torchaudio librosa pillow

# to pre download the models from https://hf-mirror.com once only
# - 0. install wget, jq for your git-bash
# - 1. locate to the the preparation folder, e.g: C:\data\github\aiapp\preparation
# - 2. change the download_to_dir in hfd_models.sh if default value (~/hfd) is not suitable for you.
# - 3. start git-bash from this preparation folder
# - 4. run this: export HF_ENDPOINT="https://hf-mirror.com" && sh hfd_models_list.sh

from transformers import pipeline
import time
import os
from PIL import Image
import requests
from io import BytesIO

# switch to different HF cfgs (hf-mirror for faster download)
print(f"env for: HF_ENDPOINT={os.environ.get('HF_ENDPOINT', 'not set')}")
#os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
#os.environ["HF_HOME"] = "/hfd"
#os.environ['HF_HUB_CACHE'] = "/hfd/hub"

# allow to retest with local_model_path_dir commented w/o restart kernel
if 'local_model_path_dir' in globals() or 'local_model_path_dir' in locals():
    del local_model_path_dir

# uncomment and set correct path to use manually pre-dwonload models.
#local_model_path_dir=r"C:\Users\bill\hfd"

# start the jupyterlab with this project root dir: --notebook-dir=C:/data/github/aiapp or can set the PROJECT_HOME with this project root dir as below:
#os.environ["PROJECT_HOME"] = "C:/data/github/aiapp"
print(f"env for: PROJECT_HOME={os.environ.get('PROJECT_HOME', 'not set')}")
data_dir = os.path.join(os.environ.get("PROJECT_HOME", "."), "homeworks/week1/transformers/data")
print(f"data_dir={data_dir}")

# --------------------------
# 1. Define Tasks & Models
# --------------------------
tasks = {
    "sentiment-analysis": {
        "models": [
            "tabularisai/multilingual-sentiment-analysis",
            "ProsusAI/finbert",
            "finiteautomata/bertweet-base-sentiment-analysis",
            "boltuix/bert-emotion",
            "cardiffnlp/twitter-roberta-base-sentiment-latest",
            "distilbert/distilbert-base-uncased-finetuned-sst-2-english",
        ],
        "inputs": [
            "I love this product!",
            "This is the worst experience ever.",
            "The weather is okay, I guess."
        ]
    },
    "ner": {
        "models": [
            "dbmdz/bert-large-cased-finetuned-conll03-english",
            "Jean-Baptiste/roberta-large-ner-english"
        ],
        "inputs": [
            "Barack Obama was born in Hawaii and worked in Washington D.C.",
            "Apple is headquartered in Cupertino, California."
        ]
    },
    "question-answering": {
        "models": [
            "deepset/tinyroberta-squad2",
            "distilbert/distilbert-base-cased-distilled-squad",
        ],
        "inputs": [
            {
                "question": "Where was Barack Obama born?",
                "context": "Barack Obama was born in Hawaii."
            },
            {
                "question": "What is the capital of France?",
                "context": "France's capital is Paris."
            }
        ]
    },
    "summarization": {
        "models": [
            "Falconsai/text_summarization",
            "google-t5/t5-base",
        ],
        "inputs": [
            "The Hubble Space Telescope is a large telescope in space. It was launched in 1990 and has taken stunning images of distant galaxies. Scientists use it to study the universe.",
            "Renewable energy sources like solar and wind are becoming more popular. They reduce reliance on fossil fuels and lower carbon emissions."
        ]
    },
    "audio-classification": {
        "models": [
            "MIT/ast-finetuned-audioset-10-10-0.4593",
            "superb/hubert-base-superb-er",
        ],
        "inputs": [data_dir + "/audio/mlk.flac"] 
    },
    "automatic-speech-recognition": {
        "models": [
            "openai/whisper-tiny",
            "openai/whisper-small",
        ],
        "inputs": [data_dir + "/audio/mlk.flac"] 
    },
    "image-classification": {
        "models": [
            "Falconsai/nsfw_image_detection",
            "google/vit-base-patch16-224",
        ],
        "inputs": [data_dir + "/image/cat_dog.jpg"] 
    },
    "object-detection": {
        "models": [
            "facebook/detr-resnet-50",
            "microsoft/conditional-detr-resnet-50",
        ],
        "inputs": [data_dir + "/image/cat_dog.jpg"] 
    }
}

# --------------------------
# 2. Helper Functions
# --------------------------
def load_pipeline(task, model_name):
    """Load a pipeline for a specific task and model."""
    try:
        return pipeline(task, model=model_name)
    except Exception as e:
        print(f"⚠️ Error loading {task} model '{model_name}': {e}")
        return None

def run_task(pipeline_fn, inputs, task_name):
    """Run inference and measure latency."""
    if pipeline_fn is None:
        return [{"error": "Model not loaded"} for _ in inputs]
    
    results = []
    latencies = []
    for input_data in inputs:
        try:
            start_time = time.time()
            output = pipeline_fn(input_data)
            latency = time.time() - start_time
            latencies.append(latency)
            results.append(output)
        except Exception as e:
            latencies.append(0)
            results.append({"error": str(e)})
    avg_latency = sum(latencies) / len(latencies) if latencies else 0
    return results, avg_latency

def check_local_model_path():
    """
    Checks if the local model path is present and accessible.
    """
    try:
        # Check if variable exists
        if 'local_model_path_dir' not in globals() and 'local_model_path_dir' not in locals():
            #print("\n⚠️ local_model_path_dir variable is not defined")
            return False
        
        # Check if variable is None or empty string
        if not local_model_path_dir:
            print("\n⚠️ local_model_path_dir is empty or None")
            return False
            
        # Check if path exists
        if not os.path.exists(local_model_path_dir):
            print(f"\n⚠️ Local model path does not exist: {local_model_path_dir}")
            return False
            
        return True
    except Exception as e:
        print(f"\n⚠️ Error while checking local_model_path_dir: {str(e)}")
        return False

# --------------------------
# 3. Run Comparisons
# --------------------------
if check_local_model_path():
    print(f"using pre-downloaded models, local_model_path_dir={local_model_path_dir}")
else:
    print(f"not using pre-downloaded models.")
 
all_results = {}
for task_name, task_data in tasks.items():
    print(f"🔍 Task: {task_name.upper()}")
    all_results[task_name] = {}
    
    for model_name in task_data["models"]:
        print(f"   → Model: {model_name}")
        # check to see if use local offline models
        if check_local_model_path():
            model_path = os.path.join(local_model_path_dir, model_name)
            pipeline_fn = load_pipeline(task_name, model_path)
        else:
            pipeline_fn = load_pipeline(task_name, model_name)
 
        inputs = task_data["inputs"]
        
        results, avg_latency = run_task(pipeline_fn, inputs, task_name)
        all_results[task_name][model_name] = {
            "outputs": results,
            "avg_latency_sec": avg_latency
        }
        
        # Print sample output (first input only)
        if results and "error" not in results[0]:
            print(f"      Sample Output: {results[0]}")
        else:
            print(f"      Error: {results[0].get('error', 'Unknown error')}")

# --------------------------
# 4. Display Summary (Optional)
# --------------------------
for task_name, task_results in all_results.items():
    print(f"\n📊 Summary for {task_name}:")
    print("-" * 80)
    
    # Calculate maximum column width
    max_model_len = max(len(model) for model in task_results.keys()) if task_results else 0
    max_model_len = max(max_model_len, 15)  # Minimum 15 characters width
    
    # Print table header
    header = f"{'Model':<{max_model_len}} | {'Avg Latency':^12} | {'Sample Output'}"
    print(header)
    print("-" * len(header))
    
    for model_name, data in task_results.items():
        latency = data["avg_latency_sec"]
        sample_output = data["outputs"][0] if data["outputs"] else "N/A"
        
        # Format output
        model_str = model_name.ljust(max_model_len)
        latency_str = f"{latency:.4f}s".center(12)
        sample_str = str(sample_output)[:100] + ("..." if len(str(sample_output)) > 100 else "")
        
        print(f"{model_str} | {latency_str} | {sample_str}")



env for: HF_ENDPOINT=not set
env for: PROJECT_HOME=not set
data_dir=.\homeworks/week1/transformers/data
using pre-downloaded models, local_model_path_dir=C:\Users\bill\hfd
🔍 Task: SENTIMENT-ANALYSIS
   → Model: tabularisai/multilingual-sentiment-analysis


Device set to use cpu


      Sample Output: [{'label': 'Very Positive', 'score': 0.49209871888160706}]
   → Model: ProsusAI/finbert


Device set to use cpu


      Sample Output: [{'label': 'neutral', 'score': 0.8351427316665649}]
   → Model: finiteautomata/bertweet-base-sentiment-analysis


...

      Sample Output: [{'label': 'collie', 'score': 0.2729230523109436}, {'label': 'papillon', 'score': 0.1813911646604538}, {'label': 'Border collie', 'score': 0.13854974508285522}, {'label': 'Japanese spaniel', 'score': 0.0916733592748642}, {'label': 'Shetland sheepdog, Shetland sheep dog, Shetland', 'score': 0.07215303927659988}]
🔍 Task: OBJECT-DETECTION
   → Model: facebook/detr-resnet-50


Some weights of the model checkpoint at C:\Users\bill\hfd\facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


      Sample Output: [{'score': 0.9984685778617859, 'label': 'cat', 'box': {'xmin': 78, 'ymin': 57, 'xmax': 309, 'ymax': 371}}, {'score': 0.9890327453613281, 'label': 'dog', 'box': {'xmin': 279, 'ymin': 20, 'xmax': 482, 'ymax': 416}}]
   → Model: microsoft/conditional-detr-resnet-50


Device set to use cpu


      Sample Output: [{'score': 0.7572169899940491, 'label': 'cat', 'box': {'xmin': 76, 'ymin': 55, 'xmax': 312, 'ymax': 372}}, {'score': 0.6198723316192627, 'label': 'dog', 'box': {'xmin': 274, 'ymin': 19, 'xmax': 484, 'ymax': 415}}]

📊 Summary for sentiment-analysis:
--------------------------------------------------------------------------------
Model                                                      | Avg Latency  | Sample Output
-----------------------------------------------------------------------------------------
tabularisai/multilingual-sentiment-analysis                |   0.1058s    | [{'label': 'Very Positive', 'score': 0.49209871888160706}]
ProsusAI/finbert                                           |   0.0530s    | [{'label': 'neutral', 'score': 0.8351427316665649}]
finiteautomata/bertweet-base-sentiment-analysis            |   0.0798s    | [{'label': 'POS', 'score': 0.9927709698677063}]
boltuix/bert-emotion                                       |   0.0244s    | [{'lab